### 특정 키워드로 유튜브 검색 후 영상 제목과 조회수를 크롤링 해보자.
- 썸네일 크기가 작은 영상들 제외하고 메인 영상들만 수집하기

In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pandas as pd
import time

In [3]:
driver = wb.Chrome()
driver.maximize_window()
keyword = input('what do you want to searh? :')
driver.get(f'https://www.youtube.com/results?search_query={keyword}')

what do you want to searh? : 뉴스


### 메인 영상 제목 가져오기
- 작은 영상들, 쇼츠 말고 메인 영상

In [79]:
videos = driver.find_elements(By.CSS_SELECTOR,'#video-title > yt-formatted-string')
print('number of videos: ', len(videos))
for i in videos:
    print(i.text)

number of videos:  20
2025년 7월 10일 (목) 오늘의 주요 뉴스 #Shorts
민주당 "체포동의안 오면 다 가결 시킬 것"‥ #Shorts (MBC뉴스)
[속보] 재구속 윤 전대통령 '내란재판' 불출석 - [MBC 뉴스속보] MBC뉴스 2025년 07월 10일
윤 전 대통령 구속영장 발부 - "증거 인멸 염려" - [LIVE] MBC 뉴스특보 2025년 07월 10일
[풀영상] 뉴스광장 : 윤석열 전 대통령 재구속…“증거인멸 염려” - 2025년 7월 10일(목) / KBS
'321호' 법정의 윤석열‥다시 '구속' 기로에 - [LIVE] MBC 뉴스데스크 2025년 07월 09일
SBS 모닝와이드 | 7/10(목) - 윤석열 전 대통령 재구속…3대 특검 수사 본격화 外
넉 달 만에 재구속‥"외환 혐의 등 수사 탄력" - [LIVE] MBC 뉴스투데이 2025년 7월 10일
"발부하면 법원 쳐들어간다"…뙤약볕 아래 "기각!" 격앙 / JTBC 뉴스룸
[뉴스쏙] 이러다 40도 간다…"더워서 못산다 출근 안할란다"｜'대기 불안정' 기습 폭우도 예보…역대급 무더위에 여기저기 난리통 /연합뉴스TV(YonhapnewsTV)
[윤석열 구속영장심사] 7월 9일 (수) JTBC 뉴스특보 풀영상 / JTBC News
이 시각 윤석열 전대통령 구속영장 심사 상황..尹 '운명의 날' 재구속 기로 - [끝까지LIVE] MBC 중계방송 2025년 07월 09일
[오늘 이 뉴스] 오늘 밤은 감방? 자택? 기로..영장심사 딱 맞춰 나타난 尹 (2025.07.09/MBC뉴스)
🔴LIVE '윤 구속' 반대 집회 현장 | "영장 기각돼야 .. 보수 지지자 4천 명 모인다" | SBS 실시간 라이브
[속보] 김건희 특검 "김여사 집사 4월 출국..여권 무효화 조치" - [현장PLAY] MBC뉴스 2025년 07월 09일
[오늘 이 뉴스] "아직도 헌법 위에 있는 줄"..'尹 태도' 본 변호사 '분노' (2025.07.09/MBC뉴스)
[속보] 법원, 윤석열 전 대통령 구속

### 조회수 정보 가져오기
- '시청 중' 정보도 같이!

In [15]:
views = driver.find_elements(By.CSS_SELECTOR,'#metadata-line > span:nth-child(3)')
print('number of views: ', len(views))
for i in views:
    str = i.text
    if str.contains('시청 중'):
        str = str.rstrip('시청 중')
    print(str)

number of views:  40


AttributeError: 'str' object has no attribute 'contains'

### 유튜브 웹 페이지의 특성상 검색이나 검색 시점에 따라서 출력되는 영상의 포맷이나 형태가 다 달라짐(변동성이 매우 큰 웹 페이지)
- 메인 영상과 주요 뉴스와 쇼츠 출력 형태가 다 다른데 영상 전체를 감싸는 태그의 내주 구조가 동일한
- 따라서 정확하게 원하는 부분에만 접근하기 위해서는 3종류의 영상의 차이가 발생하는 부분을 잘 찾아내는 것이 중요
- 메인 영상: ytd-video-renderer
- 주요 영상: ytd-grid-video-renderer
- 쇼츠: ytd-reel-item-renderer

### while문으로 스크롤 해서 더 많은 제목과 조회수 정보를 가져오고 정확히 영상 50개의 데이터만 출력하기

In [17]:
videos_list_html, views_list_html = [],[]

while len(videos_list_html) < 50:
    body = driver.find_element(By.TAG_NAME,'body')
    body.send_keys(Keys.END)
    time.sleep(3)
    
    videos = driver.find_elements(By.CSS_SELECTOR,'#video-title > yt-formatted-string')
    views = driver.find_elements(By.CSS_SELECTOR,'#metadata-line > span:nth-child(3)')

    # 영상 제목 가져오기
    new_videos = [v.text for v in videos]
    new_views = [v.text for v in views]
    
    # 누적 저장 (중복 제거)
    for v in new_videos:
        if v not in videos_list_html and v.strip() != '':
            videos_list_html.append(v)

    for v in new_views:
        if v not in views_list_html and v.strip() != '':
            views_list_html.append(v)


# 50개까지만 잘라서 리스트로 생성
videos_list = videos_list_html[:50]
views_list = views_list_html[:50]

print('total count of videos: ', len(videos_list))
print('total count of views: ', len(views_list))

total count of videos:  50
total count of views:  45


In [19]:
yt_df = pd.DataFrame({'video title':videos_list,'views':views_list}, index = range(1,len(videos_list)+1))

ValueError: Length of values (45) does not match length of index (50)

In [125]:
driver.quit()